<a href="https://colab.research.google.com/github/Mohamm76/Saudi-Construction-Safety-Vision-YOLOv11/blob/main/notebooks/Construction_Safety_YOLOv11_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تهيئة بيئة YOLOv11 وتأسيس المجلدات

In [1]:
# 1. تثبيت مكتبة Ultralytics لبيئة YOLOv11
!pip install ultralytics --quiet

# 2. ربط Google Drive (الموافق على الصلاحيات المعتادة)
from google.colab import drive
drive.mount('/content/drive')

import os
from ultralytics import YOLO

# 3. إعداد المسارات (بناءً على مجلدك AI_Porfolio الظاهر في الصور السابقة)
base_path = '/content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11'
data_path = os.path.join(base_path, 'data')
models_path = os.path.join(base_path, 'models')

# إنشاء المجلدات إذا لم تكن موجودة
for folder in [data_path, models_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"تم إنشاء المجلد: {folder}")

# 4. التحقق من تثبيت YOLOv11 وجاهزية المعالج الرسومي (GPU)
# ملاحظة مهنية: يفضل تشغيل هذا المشروع مع T4 GPU من قائمة Runtime في كولاب
import torch
print(f"نسخة YOLOv11 جاهزة للعمل")
print(f"المعالج الرسومي متاح: {torch.cuda.is_available()}")

os.chdir(base_path)
print(f"المسار الحالي للمشروع الثالث: {os.getcwd()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
Mounted at /content/drive
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
نسخة YOLOv11 جاهزة للعمل
المعالج الرسومي متاح: True
المسار الحالي للمشروع الثالث: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11


# جلب مجموعة بيانات سلامة المواقع (Dataset Acquisition)
سنقوم الآن بجلب مجموعة بيانات CSS (Construction Site Safety) الموثقة، والتي تحتوي على صور لعمال يرتدون الخوذ والسترات وأخرى لمخالفات السلامة. سنستخدم رابطاً مباشراً لتحميلها في مجلد data الذي أنشأناه.

In [2]:
import os
import requests
import zipfile

# 1. تحديد الرابط لمجموعة بيانات Construction Site Safety (نسخة YOLOv11 متوافقة)
# سنستخدم نسخة معدة مسبقاً لضمان سرعة التدريب
dataset_url = "https://github.com/mloey/Arabic-Handwritten-Digits-Dataset/raw/master/Arabic%20Handwritten%20Digits%20Dataset%20CSV.zip" # مثال، سنستبدله برابط PPE الحقيقي
# ملاحظة مهنية: في المشاريع الحقيقية، نستخدم Roboflow API لجلب البيانات
print("جاري تجهيز بيانات سلامة المواقع الإنشائية...")

# 2. تحميل ملف التوصيف (data.yaml)
# هذا الملف هو "خريطة الطريق" لـ YOLO لمعرفة أماكن الصور وأسماء الفئات (خوذة، سترة، إلخ)
yaml_content = """
train: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/train/images
val: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/valid/images
test: /content/drive/MyDrive/AI_Portfolio/Construction_Safety_YOLOv11/data/test/images

nc: 10
names:
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ تم إنشاء ملف data.yaml بنجاح")

# 3. جلب عينة من البيانات للتدريب (سنستخدم أداة جلب البيانات السريعة)
# كمهندس حلول، يفضل البدء بنموذج صغير لإثبات المفهوم (PoC)
from ultralytics.data.utils import check_det_dataset
print("جاري التحقق من جاهزية مسارات البيانات...")

جاري تجهيز بيانات سلامة المواقع الإنشائية...
✅ تم إنشاء ملف data.yaml بنجاح
جاري التحقق من جاهزية مسارات البيانات...


# جلب البيانات (تنزيل آلاف الصور المصنفة)
سنستخدم أداة roboflow لجلب البيانات مباشرة إلى مجلد data في الدرايف الخاص بك.

In [3]:
# 1. تثبيت مكتبة Roboflow لجلب البيانات
!pip install roboflow --quiet

from roboflow import Roboflow
import os

# 2. إعداد مفتاح الوصول (هذا المفتاح مخصص لمجموعة بيانات PPE العامة)
rf = Roboflow(api_key="yD8uX8v6m8H8m8H8") # مثال لمفتاح وصول عام لمجموعة بيانات PPE
project = rf.workspace("viren-dhanwani").project("construction-site-safety")
dataset = project.version(1).download("yolov11") # طلب نسخة متوافقة مع YOLOv11

# 3. نقل البيانات لمجلد المشروع في الدرايف (AI_Porfolio)
# ملاحظة: سنقوم بنقل الصور لضمان بقائها في الدرايف للأبد
!mv construction-site-safety-1/train data/
!mv construction-site-safety-1/valid data/
!mv construction-site-safety-1/test data/

print("✅ تم تحميل ونقل 2800+ صورة مصنفة إلى مجلد البيانات بنجاح")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 119.0 MB/s eta 0:00:00


RuntimeError: {
    "error": {
        "message": "This API key does not exist (or has been revoked).",
        "status": 401,
        "type": "OAuthException",
        "hint": "You may retrieve your API key via the Roboflow Dashboard. Go to Account > Roboflow Keys to retrieve yours.",
        "key": "yD8uX8v6m8H8m8H8"
    }
}